In [1]:
using Pkg
Pkg.activate(".")


  Activating project at `~/Research/SISAP2023/sisap23-laion-challenge-example/demo`


In [2]:
#=Pkg.add([
        PackageSpec(name="SimilaritySearch"),
        PackageSpec(name="TextSearch"),
        PackageSpec(name="JLD2"),
        PackageSpec(name="CSV"),
        PackageSpec(name="Parquet2"),
        PackageSpec(name="DataFrames"),
        PackageSpec(name="HypertextLiteral"),
])
=#
# Pkg.instantiate()

In [3]:
using SimilaritySearch, TextSearch, Parquet2, DataFrames, CSV, HypertextLiteral, JLD2, LinearAlgebra
using Downloads: download


In [4]:
dbname = "laion2B-en-clip768-n=10M.h5"
indexname = "index.laion2b-en-clip768-n=10M.jld2"

"index.laion2b-en-clip768-n=10M.jld2"

In [5]:
db = jldopen(dbname) do f
    X = Float32.(f["emb"])
    for c in eachcol(X)
        normalize!(c)
    end
    
    StrideMatrixDatabase(X)
end

StrideMatrixDatabase{StrideArraysCore.StrideArray{Float32, 2, (1, 2), Tuple{Int64, Int64}, Tuple{Nothing, Nothing}, Tuple{Static.StaticInt{1}, Static.StaticInt{1}}, Matrix{Float32}}}(Float32[-0.0019801862 0.0572678 … -0.020251254 0.026359847; 0.043159664 0.05241408 … -0.0052154227 0.068279326; … ; -0.009286653 0.018865405 … 0.02823272 0.023171648; -0.024937373 -0.008127693 … -0.025043186 0.02204281])

In [6]:
if isfile(indexname)
    G, _ = loadindex(indexname, db; staticgraph=true)
else
    dist = NormalizedCosineDistance()
    G = SearchGraph(; db, dist, verbose=false)
    index!(G)
    optimize!(G, MinRecall(0.9))
    saveindex(indexname, G; store_db=false)
end

(SearchGraph{NormalizedCosineDistance, StrideMatrixDatabase{StrideArraysCore.StrideArray{Float32, 2, (1, 2), Tuple{Int64, Int64}, Tuple{Nothing, Nothing}, Tuple{Static.StaticInt{1}, Static.StaticInt{1}}, Matrix{Float32}}}, SimilaritySearch.AdjacencyLists.StaticAdjacencyList{UInt32}, BeamSearch}
  dist: NormalizedCosineDistance NormalizedCosineDistance()
  db: StrideMatrixDatabase{StrideArraysCore.StrideArray{Float32, 2, (1, 2), Tuple{Int64, Int64}, Tuple{Nothing, Nothing}, Tuple{Static.StaticInt{1}, Static.StaticInt{1}}, Matrix{Float32}}}
  adj: SimilaritySearch.AdjacencyLists.StaticAdjacencyList{UInt32}
  hints: Array{Int32}((165,)) Int32[785, 845, 1416, 2220, 2504, 2626, 2686, 2838, 2954, 3026  …  20115, 20143, 20146, 20295, 20480, 20500, 20518, 20520, 20564, 20759]
  search_algo: BeamSearch
  len: Base.RefValue{Int64}
  verbose: Bool true
, nothing)

In [7]:
size(db.matrix)

(768, 10120191)

In [8]:
#jldopen(f->f["pca96"], "laion2B-en-pca96-n=10M.h5")

In [9]:
D = DataFrame(Parquet2.Dataset("meta-10M.parquet"), copycols=false)

Row,caption,NSFW,LICENSE,url
,String,String,String?,String
1,ONA Prince Street Camera Messenger Bag - Field Tan,UNLIKELY,?,https://cdn.shopify.com/s/files/1/0253/1179/products/TBPhotography-6754_small.jpg?v=1571438581
2,"One bed - three looks! You can keep the bones of your bedroom the same - your bed and major storage pieces, and completely transform the look of your the space with a switch of bedding, lighting and accessories.",UNLIKELY,?,https://i.pinimg.com/236x/e6/2d/ce/e62dce0011d491e2eba1b19e03ff33e3--summer-bedroom-ikea-bedroom.jpg
3,White flag at end of stick held by an arm against a partly cloudy sky.,UNLIKELY,?,https://g.foolcdn.com/editorial/images/472551/surrender-gettyimages-160054088_large.jpg
4,Our Complete Ikea Kitchen Remodel 8 Most Helpful Ideas,UNLIKELY,?,https://i1.wp.com/www.apieceofrainbow.com/wp-content/uploads/2019/07/Farmhouse-IKEA-Kitchen-Remodel-before-after-DIY-small-kitchen-design-remodeling-renovation-colorful-Modern-bohemian-boho-anthropologie-home-apieceofrainbow-1e.jpg?q=w%3D39%26strip%3Dall%26quality%3D24
5,Durham Bulls Seasonal Job Fair,UNLIKELY,?,https://www.capitolbroadcasting.com/wp-content/uploads/2015/01/job-fair-seasonal-150x150.jpg
6,"32 Living Room Library Design, Library/Living Room Foxcroft ... Modern Home Office Design Ideas Best 25 Offices For Small Space Interior Library Pictures Mens Study Room Webbkyrkancom Simple Nice With Dark Wooden Table Study Rooms Ideas On Pinterest Desk Families It Decorating Entrancing Home Office",UNLIKELY,?,https://t0.gstatic.com/images?q=tbn:ANd9GcRKpOQU_OhubsM65QRYDIVFPw2X6K_WBbzGnRyp-06_fpBKc_8n
7,Cover for The judge's list,UNLIKELY,?,https://contentcafe2.btol.com/ContentCafe/Jacket.aspx?UserID=NPI86627&Password=CC59937&Return=T&Type=S&Value=0385546025
8,Chapeaux Paintings - Sunday Hat in Sint Maarten by Patty Meotti,UNLIKELY,?,http://images.fineartamerica.com/images-small/sunday-hat-in-sint-maarten-patty-meotti.jpg
9,"""""""Down the Road"""" original fine art by Jo Allebach""",UNLIKELY,?,http://fineartimages.dailypaintworks.com/39c6bbb0-691c-4dc9-8aa3-a0e94eec6d5f.jpg?imageVersion=


In [22]:
invfilename = "invfile-captions-laion2B-en-10M.jld2"
# rm(invfilename)
if isfile(invfilename)
    invfile = load(invfilename, "invfile")
else
    invfile = BM25InvertedFile(TextConfig(), rand(D.caption, 300_000)) do t
        3 <= t.ndocs <= 10_000
    end

    @time append_items!(invfile, D.caption)
    jldsave(invfilename; invfile)
end

441.404738 seconds (426.95 M allocations: 103.929 GiB, 6.37% gc time, 0.50% compilation time)


In [25]:
function show_results(res)
    L = []

    for (i, p) in enumerate(res)
        r = D[p.id, :]

        push!(L, @htl("""
        <div style="width: 12%; display: inline-block; vertical-align: top; ">
                <img src="$(r.url)" style="width: 95%;"/>
                
                <span style="background-color: rgb(200, 200, 222);">$i - id: $(p.id) - w: $(round(p.weight; digits=2))</span>
                <span>$(r.caption)</span>
        </div>
            """))
    end

    @htl("""
        <div>$L</div>
    """)
end

function search_text(invfile::BM25InvertedFile, qtext::AbstractString, k)
    @time R = search(invfile, qtext, getknnresult(k))
    show_results(R.res)
end

search_text(qtext::AbstractString, k=16) = search_text(invfile, qtext, k)

function search_emb(G::SearchGraph, q, k)
    @time R = search(G, q, getknnresult(k))
    show_results(R.res)
end

search_emb(q, k=16) = search_emb(G, q, k)

search_emb (generic function with 3 methods)

In [27]:
search_text("pico de gallo")

  0.111614 seconds (1.50 M allocations: 26.317 MiB, 36.27% compilation time)


<div><div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://i.pinimg.com/736x/5f/4c/7a/5f4c7a0da0f4fde5dce67bbcdbb269ba.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">1 - id: 30949 - w: -61.81</span>
        <span>Fresh Pico de Gallo</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://cdnimages.logicommerce.cn/985//0081-Pico-de-gallo-recipe.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">2 - id: 1300988 - w: -61.81</span>
        <span>Pico de gallo recipe (0081)</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://thefarmgirlgabs.com/wp-content/uploads/2017/05/Pineapple-Pico-de-Gallo-feature5-300x200.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">3 - id: 2741264 - w: -61.81</span>
        <span>Pineapple Pico de Gallo</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://resources.mynewsdesk.com/image/upload/t_next_gen_span3_padding_4_3/x1l3hpkcskxrdweaxibw.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">4 - id: 9940470 - w: -61.81</span>
        <span>Receptbild Grilled Scalops with Pico de Gallo</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://t1.ftcdn.net/jpg/00/44/16/24/160_F_44162414_1d50lodrg8gW23SzXLT2OijpqtOVKIJC.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">5 - id: 1036885 - w: -60.26</span>
        <span>Guacamole, Pico de Gallo and Avocados</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://media.istockphoto.com/photos/pico-de-gallo-sauce-picture-id529684436?k=6&amp;amp;m=529684436&amp;amp;s=612x612&amp;amp;w=0&amp;amp;h=RFhm_ZUB_HDmG0m_Kx8pxEPuP84GSLlKSeXeHezhYMI=" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">6 - id: 2440563 - w: -60.26</span>
        <span>Salsa Pico De Gallo stock photo</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://i0.wp.com/bitsandbitesblog.com/wp-content/uploads/2016/06/IMG_5737Edited.jpg?resize=150%2C150" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">7 - id: 4172825 - w: -60.26</span>
        <span>Sweet and Spicy Pico De Gallo</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://lh3.ggpht.com/vWIpmJXWttDd0CIRW5jqNYUexPNs71nx6MOb5BZUigRehzQMJzCSTYHgy32VgeijyVskq8q488EO1gON4k0W2UY=s230-c-e365" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">8 - id: 8211379 - w: -60.26</span>
        <span>Pico de Gallo Chicken Quesadillas</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://i2.wp.com/gimmedelicious.com/wp-content/uploads/2019/11/Pico-De-Gallo-Pin.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">9 - id: 834474 - w: -58.84</span>
        <span>Pico De Gallo (Fresh Tomato Salsa)</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.traderjoesreviews.com/wp-content/uploads/2014/11/picodegallosalsa-300x174.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">10 - id: 2863212 - w: -58.84</span>
        <span>Trader Joe&apos;s Pico de Gallo Salsa</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://carolinescookbook.co.uk/wp-content/uploads/2015/09/picodegallo-960x640.jpg" style=

In [28]:
search_emb(database(G, 1036885))

  0.006014 seconds (5 allocations: 4.297 KiB)


<div><div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://t1.ftcdn.net/jpg/00/44/16/24/160_F_44162414_1d50lodrg8gW23SzXLT2OijpqtOVKIJC.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">1 - id: 1036885 - w: 0.0</span>
        <span>Guacamole, Pico de Gallo and Avocados</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://thumbs.dreamstime.com/t/avocado-guacamole-18874450.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">2 - id: 1915910 - w: 0.08</span>
        <span>Avocado and Guacamole Stock Photo</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://static8.depositphotos.com/1034635/1034/i/110/depositphotos_10342224-Guacamole-and-nachos.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">3 - id: 9287635 - w: 0.11</span>
        <span>Guacamole and nachos — Stock Photo #10342224</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://st.depositphotos.com/1692343/4976/i/170/depositphotos_49761525-Organic-Raw-Green-Avocados.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">4 - id: 1767344 - w: 0.11</span>
        <span>Organic Raw Green Avocados — 图库照片</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://static6.depositphotos.com/1068854/556/i/110/depositphotos_5563043-Avocado.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">5 - id: 903235 - w: 0.11</span>
        <span>Avocado - Foto de Stock</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://thumbs.dreamstime.com/t/guacamole-18453886.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">6 - id: 2330868 - w: 0.11</span>
        <span>Guacamole Royalty Free Stock Image</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://thumbs.dreamstime.com/t/guacamole-bowl-tasty-chips-fresh-ingredients-side-30842370.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">7 - id: 2815496 - w: 0.12</span>
        <span>Guacamole Bowl. Tasty guacamole bowl with chips and fresh ingredients on the side Stock Photo</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://healthylnb.com/wp-content/uploads/2015/03/avocado-health-300x160.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">8 - id: 1148748 - w: 0.12</span>
        <span>avocado health</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.babyboomster.com/wp-content/uploads/2014/07/Avocado-sm.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">9 - id: 9023346 - w: 0.12</span>
        <span>Avocado - Best Snack Foods for Baby Boomers babyboomster.com</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://myvalleyschools.org/wp-content/uploads/2019/05/avocado-2133723_960_720.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">10 - id: 4905206 - w: 0.12</span>
        <span>A avacado is sliced in half and sitting in front of a bowl heart disease</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://media.indiatimes.in/media/photogallery/2013/Oct/1767437_1437672005_124x124.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);

In [29]:
search_text("picasso")

  0.001013 seconds (15.18 k allocations: 288.320 KiB)


<div><div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://tse1-2.mm.bing.net/th?id=OIP.c7Jo90ZDr3Xn1h_4CW15hQHaI4" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">1 - id: 276649 - w: -24.32</span>
        <span>picasso paintings of therese best 25 pablo picasso ideas on picasso</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://cdn.shopify.com/s/files/1/2584/1070/products/picasso-and-picasso-petrina-easton_eb0d4d2d-4baa-4085-a1e3-6cfb126829cc_2048x2048.jpg?v=1515713857" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">2 - id: 2910582 - w: -24.01</span>
        <span>Picasso And Picasso - Phone Case</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.michelfillion.com/img/photos/picasso_122b.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">3 - id: 4005876 - w: -24.01</span>
        <span>PICASSO : picasso-profil-homme-etching</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://imageprocessor.digital.vistaprint.com/crop/3,0,3241x3241/width/1100/maxWidth/2000/http://uploads.documents.cimpress.io/v1/uploads/609e20e2-f716-4f18-a385-383809c40c76~110/original?tenant=vbu-digital" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">4 - id: 5393202 - w: -23.84</span>
        <span>Picasso, Picasso&apos;s Dogs, Artists, Artists&apos; Dogs, Beret, Picasso&apos;s Style of Dress, Anthropomorphism, Picasso&apos;s Blue Period</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://tse1.mm.bing.net/th?id=OIP.MgBbz0gtW3iJNRhI6dfvOwHaFj" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">5 - id: 1088261 - w: -23.6</span>
        <span>picasso paintings guernica meaning guernica by pablo picasso</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://ecx.images-amazon.com/images/P/1573229539.01._SL160_SCLZZZZZZZ__.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">6 - id: 1548318 - w: -23.6</span>
        <span>Picasso, Marina: Picasso, My Grandfather</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://dcqohhuh7tdzn.cloudfront.net/9780/9783829029780_medium.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">7 - id: 2720965 - w: -23.6</span>
        <span>Picasso : Life and Work, Picasso, Pablo, 3829029780</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.galerie-f.de/bilder/09_05_2011_6/thumbs/2.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">8 - id: 4721206 - w: -23.6</span>
        <span>Pablo Picasso: Picasso At Pace/Columbus</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://ctl.s6img.com/society6/img/ZDYcvOtZRTTcsrhX3jIdPIb63EQ/w_200,h_200/bags/small/close/~artwork/s6-0019/a/6939387_15826991.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">9 - id: 9560756 - w: -23.6</span>
        <span>picasso Tote Bags featuring Picasso by Mitja Bokun</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://tse1.mm.bing.net/th?id=OIP.SzAqg7oi-XALwycDk_OcVQHaKU" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">10 - id: 1415158 - w: -23.21</span>
        <span>picasso paintings buy picasso paintings 2018 dr</span>
</div>
    <div style

In [30]:
search_emb(database(G, 276649))

  0.003655 seconds (5 allocations: 4.297 KiB)


<div><div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://tse1-2.mm.bing.net/th?id=OIP.c7Jo90ZDr3Xn1h_4CW15hQHaI4" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">1 - id: 276649 - w: -0.0</span>
        <span>picasso paintings of therese best 25 pablo picasso ideas on picasso</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://cdn.gallerix.asia/x/src/news/2018/Feb/annabelle.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">2 - id: 10106082 - w: 0.07</span>
        <span>&quot;&quot;&quot;&quot;&quot;&quot;&quot;Annabelle&quot;&quot;&quot;&quot; Picasso - renommer l&apos;image&quot;&quot;&quot;</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://arthive.net/res/media/img/orig/work/f82/190422.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">3 - id: 120995 - w: 0.09</span>
        <span>Pablo Picasso. Jacqueline with flowers</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://cdn.shopify.com/s/files/1/0030/0816/7981/products/C-0087_pablo_picasso_woman_hat_tube_XL_preview_2.jpg?v=1560457398" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">4 - id: 8241714 - w: 0.09</span>
        <span>Pablo Picasso - Woman in Hat and Fur Collar</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://www.galeriedada.com/img_big/Pic25.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">5 - id: 2912445 - w: 0.11</span>
        <span>Head of a Woman and Stars</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://images.squarespace-cdn.com/content/v1/53f353f4e4b0dbe1ff73ced2/1422512239182-DY3BI5Z51HWHD1JJ43SL/ke17ZwdGBToddI8pDm48kIkSVuU5FBEttbBmb1_WZWpZw-zPPgdn4jUwVcJE1ZvWEtT5uBSRWt4vQZAgTJucoTqqXjS3CfNDSuuf31e0tVF4v0VhzdbKbwhHmsdGUP7ETZpks4BHXsHNS63VxYA0K7vSAxncsgLUx_FOHuXVbfc/17+Picasso+Portrait+of+Dora+Maar+1937.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">6 - id: 7966224 - w: 0.11</span>
        <span>Portrait of Dora Maar, 1937</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="http://www.ownmodern.com/images/monet_rothko_kandinsky_more/51gPNqB2FDL._SY576_CR0,0,400,500_.jpg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">7 - id: 5664127 - w: 0.11</span>
        <span>Pablo Picasso Portrait of Marie Therese Framed Oil Painting, 20 x 24</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://images.squarespace-cdn.com/content/57a4bc6303596ec41f3101a8/1503075806663-NLTF9EX1YPLZFUR9VOSE/Dora+Maar+by+Picasso%2C+1941?content-type=image%2Fjpeg" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">8 - id: 4522674 - w: 0.11</span>
        <span>Dora Maar by Picasso, 1941</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://p1.liveauctioneers.com/546/23202/8178368_1_x.jpg?version=1289414275&amp;amp;width=170&amp;amp;format=pjpg&amp;amp;auto=webp&amp;amp;quality=50" style="width: 95%;"/>
        
        <span style="background-color: rgb(200, 200, 222);">9 - id: 5663288 - w: 0.11</span>
        <span>PABLO PICASSO The Red Armchair Print, Limited Edition</span>
</div>
    <div style="width: 12%; display: inline-block; vertical-align: top; ">
        <img src="https://artlogic-res.cloudinary.com/w_1388,h_620,c_limit,f_auto,fl_lossy,q_auto/artlogicstorage/houkgallery/images/view/e74be8ca9d77c